In [1]:
import numpy as np

In [ ]:
# HardlyHumans/Facial-expression-detection 모델에서 분류하는 감정들 (labels = model.config.id2label 의 결과)
backbone_classes = {
    0: 'anger', 
    1: 'contempt', 
    2: 'disgust', 
    3: 'fear', 
    4: 'happy', 
    5: 'neutral', 
    6: 'sad', 
    7: 'surprise'
}

In [ ]:
# EST_data에서 라벨러가 사용한 감정 종류 (7개): ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립'] 를 백본 모델의 분류로 매핑
# 기쁨 -> happy
# 당황 -> surprise, fear로 소프트 라벨링
# 분노 -> anger, contempt, disgust로 소프트 라벨링
# 불안 -> fear
# 상처 -> sad
# 슬픔 -> sad
# 중립 -> neutral

annotation_to_backbone_map = {
    "기쁨": {"happy": 1.0},
    "당황": {"fear": 0.4, "surprise": 0.6}, # 예시
    "분노": {"anger": 0.5, "contempt": 0.3, "disgust": 0.2}, # 예시
    "불안": {"fear": 1.0},
    "상처": {"sad": 1.0},
    "슬픔": {"sad": 1.0},
    "중립": {"neutral": 1.0}
}

In [ ]:
def create_soft_label(annotation_labels): # 매핑 함수
    """
    annotation_labels: list of annotation 감정 문자열, 예: ["분노", "불안", "상처"]
    반환: numpy array, 길이 8, 각 백본 클래스에 대한 확률 분포 (soft label)
    """
    soft_label = np.zeros(len(backbone_classes), dtype=np.float32)
    # 각 annotation 감정 별 백본 클래스 가중치 누적
    for ann in annotation_labels:
        if ann in annotation_to_backbone_map:
            mapped = annotation_to_backbone_map[ann]
            for cls_name, weight in mapped.items():
                # 딕셔너리에서 클래스명에 해당하는 인덱스 찾기
                idx_list = [idx for idx, name in backbone_classes.items() if name == cls_name]
                if idx_list:
                    idx = idx_list[0]
                    soft_label[idx] += weight
        else:
            # 미등록 감정은 중립으로 처리
            neutral_idx = [idx for idx, name in backbone_classes.items() if name == 'neutral'][0]
            soft_label[neutral_idx] += 1.0

    # 확률 합이 0이면 중립으로 설정
    total = soft_label.sum()
    if total > 0:
        soft_label /= total
    else:
        neutral_idx = [idx for idx, name in backbone_classes.items() if name == 'neutral'][0]
        soft_label[neutral_idx] = 1.0

    return soft_label # np.array

In [ ]:
# 예시, 어떤 사진에 대해 세 라벨러가 ["분노", "불안", "상처"]로 라벨링을 했을 때의 소프트 라벨링
example_annotations_1 = ["분노", "불안", "상처"]
example_soft_label = create_soft_label(example_annotations_1)

print("Soft label vector (index order):")
for i, cl in enumerate(backbone_classes):
    print(f"{cl}: {example_soft_label[i]:.3f}")

Soft label vector (index order):
0: 0.167
1: 0.100
2: 0.067
3: 0.333
4: 0.000
5: 0.000
6: 0.333
7: 0.000


이후 모델 파이프라인에 넣는 예제
``` python
from PIL import Image

# anger no.1 img
img = Image.open("/workspace/Data/EST_data/img_test/anger/0h5sfbee84a3ee8e7efdd0f578f83e4df7e491dbcdab482a480a7add89daegs3z.jpg")
# result
result = pipe(img)
print(result) # 중립 61%, 놀람 34% 반환
# [{'label': 'neutral', 'score': 0.6180378198623657}, {'label': 'surprise', 'score': 0.3413133919239044}, {'label': 'sad', 'score': 0.014007982797920704}, {'label': 'contempt', 'score': 0.010649532079696655}, {'label': 'fear', 'score': 0.008162293583154678}]


'''
# 해당 파일의 json 구조
{
        "filename": "0h5sfbee84a3ee8e7efdd0f578f83e4df7e491dbcdab482a480a7add89daegs3z.jpg",
        "gender": "여",
        "age": 30,
        "isProf": "전문인",
        "faceExp_uploader": "분노",
        "bg_uploader": "공공시설/종교/의료시설",
        "annot_A": {
            "boxes": {
                "maxX": 2002.1239,
                "maxY": 1270.1625,
                "minX": 1342.259,
                "minY": 380.77808
            },
            "faceExp": "당황",
            "bg": "상업시설/점포/시장"
        },
        "annot_B": {
            "boxes": {
                "maxX": 1989.1384060232301,
                "maxY": 1280.5532041653823,
                "minX": 1339.6129150294,
                "minY": 380.80842846619595
            },
            "faceExp": "불안",
            "bg": "공공시설/종교/의료"
        },
        "annot_C": {
            "boxes": {
                "maxX": 2002.1238999999998,
                "maxY": 1270.1625,
                "minX": 1342.259,
                "minY": 380.77808000000005
            },
            "faceExp": "당황",
            "bg": "공공시설/종교/의료"
        }
    }
'''
```